In [116]:
import glob, os
from pathlib import Path
import pandas as pd
path = "C:\\Users\\Exams Office\\Desktop\\moderation_2021_22\\*"
files = glob.glob(path)

In [117]:
import warnings
from pandas.api.types import is_string_dtype
from pathlib import Path

# warnings.filterwarnings()
def normal_excel_upload(excel_file):
    try:
        df = pd.read_excel(excel_file, header=None)
    except Exception:
        print("Problem reading excel file")
    df = df.fillna('')
    for col in df.columns:
        try:
            df[col] = df[col].str.strip()
        except Exception:
            df[col] = df[col]
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except Exception:
            continue

    results_row_df = pd.DataFrame()
    reg_no_col = None

    # find reg number column
    for col in df.columns:
        df_gen = df[df[col].str.contains(pat="[0-9]{4}\/[0-9]{6}", regex=True, na=False)]

        if not df_gen.empty:
            reg_no_col = col
            results_row_df = df_gen
            break
    if reg_no_col is None:
        # print(results_row_df)
        raise Exception("Bad result formatting! Reg numbers not detected")
    results_row_df = results_row_df.iloc[:, reg_no_col:]
    results_row_df[reg_no_col] = results_row_df[reg_no_col].str.strip()
    print(results_row_df)
    
    try:
        results = results_row_df.copy()[[reg_no_col, 7]]
    except Exception:
        print(results_row_df)
        raise Exception("invalid template detected!")
    file_name = Path(excel_file).stem[:7]
    results[99] = file_name
    results[2] = results[2].str.strip()
    os.makedirs('files', exist_ok=True)
    results.to_csv(f'files/{file_name}.csv', header=False, index=False)
    return results
    

In [118]:
# normal_excel_upload(r'C:\Users\Exams Office\Desktop\moderation_2021_22\ECE 521_2021_2022_ Industrial Electronics Design.xlsx')

In [119]:
all_results = []
for file in files:
    try:
        all_results.append(normal_excel_upload(file))
    except Exception as exc:
        print(exc)
        print("problem file: ", file)
        continue
combined_res = pd.concat(all_results).reset_index(drop=True)

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


               2    3    4    5    6    7    8
11   2017/242559  ECE  5\5  NaN  NaN  NaN    B
12   2017/247306  ECE  5\5  NaN  NaN  NaN    E
13   2017/243330  ECE  5\5  NaN  NaN  NaN   FF
14   2017/247307  ECE  5\5  NaN  NaN  NaN   FF
15   2017/241523  ECE  5\5  NaN  NaN  NaN    C
16   2017/241426  ECE  5\5  NaN  NaN  NaN    B
17   2015/202051  ECE  5\5  NaN  NaN  NaN    D
18   2017/247314  ECE  5\5  NaN  NaN  NaN    E
19   2017/247316  ECE  5\5  NaN  NaN  NaN    C
20   2017/247319  ECE  5\5  NaN  NaN  NaN    B
21   2016/233496  ECE  5\5  ABS  ABS  ABS  ABS
22   2017/243337  ECE  5\5  ABS  ABS  ABS  ABS
23   2017/247322  ECE  5\5  NaN  NaN  NaN    B
24   2017/247324  ECE  5\5  NaN  NaN  NaN   FF
25   2017/247330  ECE  5\5  NaN  NaN  NaN   FF
26   2018/250680  ECE  5\5  NaN  NaN  NaN    A
27   2017/247332  ECE  5\5  NaN  NaN  NaN    B
28   2017/247336  ECE  5\5  NaN  NaN  NaN    A
29   2016/237223  ECE  5\5  NaN  NaN  NaN    C
30   2017/247339  ECE  5\5  NaN  NaN  NaN    C
31   2017/243

In [120]:
combined_res[7] = combined_res[7].apply(lambda x: "XX" if isinstance(x, str) else x)
combined_res.columns = ["reg_no", "score", "course"]
combined_res.to_csv("ALL_RESULTS.csv", index=False)
# combined_res[7][combined_res[7].apply(lambda x: isinstance(x, str))]
# combined_res